# Step 0: Download Data


# GPU-Accelerated Time Series Forecasting

**Hardware**: RTX 3070 + CUDA 13.1 + CuPy 13.3.0 + LightGBM

**7-Step Pipeline**:
1. Load data & baseline metric
2. Temporal features (lags, rolling windows)
3. Horizon-specific LightGBM models
4. PCA dimensionality reduction
5. Smoothed target encoding
6. Feature selection & interactions
7. Ensemble predictions

**Key Design**: Matrix operations (GPU) → CuPy. External libs (CPU) → NumPy.


In [1]:
# GPU Setup & Initialization

import cupy as np
import numpy as np_cpu
import sys

print(f"Python: {sys.executable}")
print(f"CuPy Version: {np.__version__}")

# Check GPU
try:
    device_count = np.cuda.runtime.getDeviceCount()
    device = np.cuda.Device(0)
    cap = device.compute_capability
    print(f"✓ GPU Ready: {device_count} device(s), Compute Capability {cap}")
except Exception as e:
    print(f"✗ GPU Error: {e}")

Python: c:\Users\anike\Desktop\ts-forecasting\.venv\Scripts\python.exe
CuPy Version: 13.6.0
✓ GPU Ready: 1 device(s), Compute Capability 86



# Step 1: Download Data


In [2]:

import os
import requests

def download_data(url, filepath):
    if os.path.exists(filepath):
        print(f"File {filepath} already exists. Skipping download.")
        return

    print(f"Downloading data from {url}...")
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded to {filepath}")
    else:
        print(f"Failed to download. Status code: {response.status_code}")

DATA_URL = "https://storage.googleapis.com/kagglesdsdata/competitions/105581/15271735/test.parquet?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1770992767&Signature=Fj%2F5LPgyVbzbph1WeG3uCJRMdqsabiq%2BKM3msHzs8y%2BXoHuAzoKKjvFDg3t8ueiPngilQtsFWg4FbUfzajf%2BDqzehOTGwRWzU6bX0xvdrrOeQusa7glpDDnF9n6C0izwzU8k%2FxFDdU7qI6vUtJLm3Yk20zfZMYx%2BuGtFrrtoTzcUx0k5ut%2Ft4OtyyBeCzpsUpCA5EjKMGbqRnB5P%2F7SCEWlwCQ7ZXL8w0kKJGCC3%2FYOqSktMDRhaeLsyP3lfCejur%2BxGfcd8hoLQWsEHOkYEw91k%2F%2BOy6vg5PkpYlQN2Exovmjg3o56VBIAZLLZhU%2BCAvtfL4X%2Bw02HrNJVKi5mhoA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.parquet"
download_data(DATA_URL, "data/test.parquet")


File data/test.parquet already exists. Skipping download.



# Step 2: Imports & Metric


In [3]:
# Utilities & Metric

import polars as pl
import warnings
import os
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from typing import Tuple, List, Dict
import requests

warnings.filterwarnings("ignore")

# GPU ↔ CPU Conversion
def gpu_to_cpu(x):
    """CuPy GPU → NumPy CPU (handles scalars + arrays)."""
    if x is None:
        return None
    try:
        # Use CuPy's .get() to explicitly convert to NumPy
        if hasattr(x, 'get'):
            return x.get()
        elif hasattr(x, 'item'):
            return x.item()
        else:
            return np_cpu.asarray(x)
    except Exception as e:
        # Fallback: ensure we return NumPy array, not CuPy
        return np_cpu.asarray(x)

def cpu_to_gpu(x):
    """NumPy CPU → CuPy GPU."""
    return np.asarray(x) if x is not None else None

# Weighted RMSE Skill Score
def weighted_rmse_score(y_true: np.ndarray, y_pred: np.ndarray, 
                        weights: np.ndarray) -> float:
    """GPU-accelerated metric. Returns Python float."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    weights = np.asarray(weights)
    
    score = 1 - np.sqrt(np.sum(weights * (y_true - y_pred) ** 2) / 
                        np.sum(weights * y_true ** 2))
    return float(score) if np.sum(weights * y_true ** 2) > 0 else 0.0

# Step 3: Load Data & Baseline (Iter A)

In [4]:

# Iteration A: The Golden Split & Metric (Polars)

def load_and_split_data(
    filepath: str = "data/test.parquet",
    target_col: str = "feature_ch",
    weight_col: str = "feature_cg",
    valid_ratio: float = 0.25,
) -> Tuple[pl.DataFrame, pl.DataFrame, list]:
    print("Loading data from", filepath)
    if not os.path.exists(filepath):
        # Create dummy data for testing if file doesn't exist
        print("Warning: Data file not found. Creating dummy data.")
        n_rows = 10000
        
        # CuPy strict typing: Must convert lists to arrays for random.choice
        codes = np.array(["A", "B"], dtype=object)
        sub_codes = np.array(["X", "Y"], dtype=object)
        categories = np.array(["1", "2"], dtype=object)
        horizons_list = np.array([1, 10, 25], dtype=int)
        
        df = pl.DataFrame({
            "id": [f"c_sc_cat_h_{i}" for i in range(n_rows)],
            "ts_index": list(range(n_rows)),  # Polars expects lists, not arrays
            "code": np.random.choice(codes, n_rows).tolist(),  # Convert GPU array to list for Polars
            "sub_code": np.random.choice(sub_codes, n_rows).tolist(),
            "sub_category": np.random.choice(categories, n_rows).tolist(),
            "horizon": np.random.choice(horizons_list, n_rows).tolist(),
            "feature_ch": np.random.randn(n_rows).tolist(),  # Target
            "feature_cg": np.random.uniform(0.5, 1.5, n_rows).tolist(), # Weight
        })
        # Add dummy features
        for i in range(10):
            df = df.with_columns(pl.lit(np.random.randn()).alias(f"feature_{i}"))
    else:
        df = pl.read_parquet(filepath)
    
    print(f"Loaded {df.height:,} rows with {len(df.columns)} columns")

    # Determine split point based on ts_index
    min_ts = df["ts_index"].min()
    max_ts = df["ts_index"].max()
    ts_range = max_ts - min_ts
    split_ts = max_ts - int(ts_range * valid_ratio)

    print(f"Time index range: {min_ts} to {max_ts}")
    print(f"Validation split at ts_index >= {split_ts}")

    # Split data using filter (lazy or eager)
    train_df = df.filter(pl.col("ts_index") < split_ts)
    valid_df = df.filter(pl.col("ts_index") >= split_ts)
    
    # Feature columns (exclude meta)
    exclude_cols = ["id", "code", "sub_code", "sub_category", target_col, weight_col, "ts_index", "horizon"]
    feature_cols = [c for c in df.columns if c not in exclude_cols]

    return train_df, valid_df, feature_cols

# Execute Iter A
train_df, valid_df, feature_cols = load_and_split_data()

# Baseline Calculation (GPU-accelerated with proper scalar handling)
y_true_gpu = cpu_to_gpu(valid_df["feature_ch"].to_numpy())
weights_gpu = cpu_to_gpu(valid_df["feature_cg"].fill_null(1.0).to_numpy())

# IMPORTANT: GPU scalars must be explicitly converted to Python scalars
train_mean_gpu = np.mean(cpu_to_gpu(train_df["feature_ch"].to_numpy()))
train_mean = float(train_mean_gpu)  # Convert GPU scalar to Python float

y_pred_baseline = np.ones_like(y_true_gpu) * train_mean
baseline_score = weighted_rmse_score(y_true_gpu, y_pred_baseline, weights_gpu)
print(f"Baseline (Mean Prediction) Score: {baseline_score:.4f}")


Loading data from data/test.parquet
Loaded 1,447,107 rows with 92 columns
Time index range: 3602 to 4376
Validation split at ts_index >= 4183
Baseline (Mean Prediction) Score: 0.3315


# Step 4: Temporal Features (Iter B)

In [5]:

# Iteration B: Temporal Feature Engineering (Polars)
# Polars makes rolling windows extremely fast and clean.

def create_temporal_features_pl(
    df: pl.DataFrame,
    feature_cols: List[str],
    group_cols: List[str] = ["code", "sub_code", "sub_category"],
    rolling_windows: List[int] = [3, 7, 14, 30],
) -> pl.DataFrame:
    print("Creating temporal features with Polars...")
    
    # Ensure sorted by ts_index within groups
    df = df.sort(group_cols + ["ts_index"])
    
    features_to_process = feature_cols[:20] if len(feature_cols) > 50 else feature_cols
    
    # Define expressions for rolling/lag features
    exprs = []
    
    for feat in features_to_process:
        # Lags
        for lag in [1, 2, 3]:
            exprs.append(pl.col(feat).shift(lag).over(group_cols).alias(f"{feat}_lag_{lag}"))
            
        # Rolling Means (Shifted by 1 to prevent leakage)
        # Polars rolling operates on the column. .shift(1) ensures we don't peek.
        for window in rolling_windows:
            # rolling_mean usage: .rolling_mean(window_size).shift(1)
            # We must use .over(group_cols) to respect groups!
            # However, rolling_mean is deprecated in favor of rolling().mean()
            # To apply over groups efficiently:
            # We can use window functions inside over()
            
            # Note: naive .rolling() inside over() can be tricky in older Polars versions, 
            # but newer versions support it well.
            # Best practice: use creating rolling columns separately if over() is complex or use window functions.
            
            # Efficient pattern: 
            # (col(feat).shift(1).rolling_mean(window)).over(group_cols)
            # Shift FIRST to prevent leakage, then roll. Wait.
            # If we shift first, then rolling window at T includes T-1, T-2... which is safe.
            # Yes.
            
            col_name = f"{feat}_roll_{window}"
            exprs.append(
                pl.col(feat)
                .shift(1)
                .rolling_mean(window_size=window, min_periods=1)
                .over(group_cols)
                .alias(col_name)
            )
            
        # Expanding Mean (Shifted)
        # Cumulative sum / Count
        # Shift(1) first
        shifted = pl.col(feat).shift(1)
        exprs.append(
            (shifted.cum_sum() / shifted.cum_count())
            .over(group_cols)
            .alias(f"{feat}_exp_mean")
            .fill_nan(0) # Handle potential division by zero
        )

    # Apply all expressions at once! Ultra fast.
    df = df.with_columns(exprs)
    
    print(f"Created {len(exprs)} temporal features")
    return df

# Combine
full_df = pl.concat([train_df, valid_df])

# Create Features
full_df = create_temporal_features_pl(full_df, feature_cols)

# Re-split
# We need to calculate split_ts again or reuse
split_ts = full_df["ts_index"].max() - int((full_df["ts_index"].max() - full_df["ts_index"].min()) * 0.25)
train_df = full_df.filter(pl.col("ts_index") < split_ts)
valid_df = full_df.filter(pl.col("ts_index") >= split_ts)

# Update feature list
current_features = [c for c in full_df.columns if c not in ["id", "code", "sub_code", "sub_category", "feature_ch", "feature_cg", "ts_index", "horizon"]]
print(f"Total features after Iter B: {len(current_features)}")


Creating temporal features with Polars...


Created 160 temporal features
Total features after Iter B: 244


# Iteration C: Weighted LightGBM

In [6]:

# Iteration C: Weighted LightGBM (Optimized GPU Pipeline)

def train_lgb_model(train_df, valid_df, features, params=None):
    if params is None:
        params = {
            "objective": "regression",
            "metric": "rmse",
            "learning_rate": 0.05,
            "num_leaves": 31,
            "verbose": -1,
            "n_jobs": -1,
            "device": "gpu"
        }
    
    # Extract to GPU arrays in one batch
    X_train_gpu = cpu_to_gpu(train_df.select(features).fill_null(0).to_numpy())
    y_train_gpu = cpu_to_gpu(train_df["feature_ch"].to_numpy())
    w_train_gpu = cpu_to_gpu(train_df["feature_cg"].fill_null(1.0).to_numpy())
    
    X_valid_gpu = cpu_to_gpu(valid_df.select(features).fill_null(0).to_numpy())
    y_valid_gpu = cpu_to_gpu(valid_df["feature_ch"].to_numpy())
    w_valid_gpu = cpu_to_gpu(valid_df["feature_cg"].fill_null(1.0).to_numpy())
    
    # Transfer to CPU only once for LightGBM
    X_train_np = gpu_to_cpu(X_train_gpu)
    y_train_np = gpu_to_cpu(y_train_gpu)
    w_train_np = gpu_to_cpu(w_train_gpu)
    
    X_valid_np = gpu_to_cpu(X_valid_gpu)
    y_valid_np = gpu_to_cpu(y_valid_gpu)
    w_valid_np = gpu_to_cpu(w_valid_gpu)
    
    train_data = lgb.Dataset(X_train_np, label=y_train_np, weight=w_train_np)
    valid_data = lgb.Dataset(X_valid_np, label=y_valid_np, weight=w_valid_np, reference=train_data)
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, valid_data],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(False)]
    )
    return model

# Train separate models for horizons
horizons = sorted(train_df["horizon"].unique().to_list())
models = {}

print("Training Horizon-specific Models (Iter C)...")
for h in horizons:
    t_h = train_df.filter(pl.col("horizon") == h)
    v_h = valid_df.filter(pl.col("horizon") == h)
    
    if t_h.height == 0 or v_h.height == 0: continue
        
    model = train_lgb_model(t_h, v_h, current_features)
    models[h] = model
    
    # Score (GPU-accelerated)
    v_preds = model.predict(v_h.select(current_features).fill_null(0).to_numpy())
    v_preds_gpu = cpu_to_gpu(v_preds)
    y_true_gpu = cpu_to_gpu(v_h["feature_ch"].to_numpy())
    weights_gpu = cpu_to_gpu(v_h["feature_cg"].fill_null(1.0).to_numpy())
    score = weighted_rmse_score(y_true_gpu, v_preds_gpu, weights_gpu)
    print(f"  Horizon {h} Score: {score:.4f}")

# Collect predictions on GPU then transfer once
preds_full = []
for h, model in models.items():
    sub_df = valid_df.filter(pl.col("horizon") == h)
    if sub_df.height > 0:
        preds = model.predict(sub_df.select(current_features).fill_null(0).to_numpy())
        temp_df = sub_df.select("id").with_columns(pl.Series(name="pred_iter_c_h", values=preds))
        preds_full.append(temp_df)

valid_df = valid_df.with_columns(pl.lit(0.0).alias("pred_iter_c"))
if preds_full:
    preds_all = pl.concat(preds_full)
    valid_df = valid_df.join(preds_all, on="id", how="left").with_columns(
        pl.col("pred_iter_c_h").fill_null(0).alias("pred_iter_c")
    )

# Overall evaluation (GPU-accelerated)
y_true_gpu = cpu_to_gpu(valid_df["feature_ch"].to_numpy())
pred_gpu = cpu_to_gpu(valid_df["pred_iter_c"].to_numpy())
weights_gpu = cpu_to_gpu(valid_df["feature_cg"].fill_null(1.0).to_numpy())
overall_score_c = weighted_rmse_score(y_true_gpu, pred_gpu, weights_gpu)
print(f"Overall Iteration C Score: {overall_score_c:.4f}")


Training Horizon-specific Models (Iter C)...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[270]	training's rmse: 0.168577	valid_1's rmse: 0.438101
  Horizon 1 Score: 0.8802
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[263]	training's rmse: 0.165978	valid_1's rmse: 0.438827
  Horizon 3 Score: 0.8805
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[172]	training's rmse: 0.190387	valid_1's rmse: 0.422422
  Horizon 10 Score: 0.8872
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[171]	training's rmse: 0.188389	valid_1's rmse: 0.43212
  Horizon 25 Score: 0.8894
Overall Iteration C Score: 0.8843


# Iteration D: PCA

In [7]:

# Iteration D: PCA (GPU-optimized)

print("Applying PCA (Iter D)...")

# Select numeric features
pca_features = [c for c in train_df.columns if c.startswith("feature_") or "_roll_" in c]
pca_features = pca_features[:50]

# Load to GPU
X_train_gpu = cpu_to_gpu(train_df.select(pca_features).fill_null(0).to_numpy())
X_valid_gpu = cpu_to_gpu(valid_df.select(pca_features).fill_null(0).to_numpy())

# GPU-accelerated standardization (all operations on GPU)
mean_gpu = np.mean(X_train_gpu, axis=0, keepdims=True)
std_gpu = np.std(X_train_gpu, axis=0, keepdims=True)
std_gpu = np.where(std_gpu == 0, 1.0, std_gpu)  # Avoid division by zero on GPU

X_train_scaled = (X_train_gpu - mean_gpu) / std_gpu
X_valid_scaled = (X_valid_gpu - mean_gpu) / std_gpu

# Transfer to CPU only for sklearn PCA
X_train_np = gpu_to_cpu(X_train_scaled)
X_valid_np = gpu_to_cpu(X_valid_scaled)

n_components = 10
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_np)
X_valid_pca = pca.transform(X_valid_np)

# Add back to DataFrames
pca_cols = [f"pca_{i}" for i in range(n_components)]
train_pca_df = pl.DataFrame(X_train_pca, schema=pca_cols)
valid_pca_df = pl.DataFrame(X_valid_pca, schema=pca_cols)

train_df = pl.concat([train_df, train_pca_df], how="horizontal")
valid_df = pl.concat([valid_df, valid_pca_df], how="horizontal")

features_d = current_features + pca_cols
print(f"PCA features added: {len(pca_cols)}")


Applying PCA (Iter D)...
PCA features added: 10


# Iteration E: Smoothed Target Encoding

In [8]:
# Iteration E: Smoothed Target Encoding (Polars Fast)

def create_smoothed_target_encoding_pl(
    df, col, target="feature_ch", weight="feature_cg", min_samples=10, smoothing=10
):
    # Sort
    df = df.sort([col, "ts_index"])
    global_mean = df[target].mean()
    
    # Polars expressions for expanding mean
    # We want: (cumsum_shift * n_shift + smooth*global) / (n_shift + smooth)
    
    # Calculate Expanding Sum and Count
    # Use over()
    
    return df.with_columns(
        (
            (pl.col(target).shift(1).cum_sum().over(col).fill_null(0) + smoothing * global_mean) 
            / 
            (pl.col(target).shift(1).cum_count().over(col).fill_null(0) + smoothing)
        ).alias(f"{col}_enc_smooth")
    )

print("Applying Smoothed Target Encoding (Iter E)...")
# Ensure both DataFrames have matching columns by selecting only those in common
cols_in_both = [c for c in train_df.columns if c in valid_df.columns]
full_df = pl.concat([train_df.select(cols_in_both), valid_df.select(cols_in_both)])

for col in ["code", "sub_code", "sub_category"]:
    full_df = create_smoothed_target_encoding_pl(full_df, col)

# Re-split
train_df = full_df.filter(pl.col("ts_index") < split_ts)
valid_df = full_df.filter(pl.col("ts_index") >= split_ts)

features_e = features_d + [f"{c}_enc_smooth" for c in ["code", "sub_code", "sub_category"]]

Applying Smoothed Target Encoding (Iter E)...


# Step 8: Feature Selection (Iter F)

In [9]:

# Iteration F: Interaction Features & Feature Selection (GPU-optimized)

print("Creating Interaction Features (Iter F)...")
new_cols = []
base_feats = [c for c in features_e if "lag" in c][:5]

for feat in base_feats:
    new_cols.append((pl.col(feat) * pl.col("horizon")).alias(f"{feat}_x_horizon"))

new_cols.append((pl.col("horizon") ** 2).alias("horizon_squared"))

train_df = train_df.with_columns(new_cols)
valid_df = valid_df.with_columns(new_cols)

interaction_feats = [f"{feat}_x_horizon" for feat in base_feats] + ["horizon_squared"]
all_candidates_f = features_e + interaction_feats

# Feature Selection with GPU acceleration
X_train_gpu = cpu_to_gpu(train_df.select(all_candidates_f).fill_null(0).to_numpy())
y_train_gpu = cpu_to_gpu(train_df["feature_ch"].to_numpy())

# GPU-accelerated correlation/variance calculations before sklearn
variances_gpu = np.var(X_train_gpu, axis=0)
X_train_np = gpu_to_cpu(X_train_gpu)
y_train_np = gpu_to_cpu(y_train_gpu)

selector = SelectKBest(score_func=f_regression, k=min(100, len(all_candidates_f)))
selector.fit(X_train_np, y_train_np)
selected_indices = selector.get_support(indices=True)
selected_features_f = [all_candidates_f[i] for i in selected_indices]
print(f"Selected {len(selected_features_f)} features from {len(all_candidates_f)} candidates")


Creating Interaction Features (Iter F)...
Selected 100 features from 263 candidates


# Iteration G: Ensemble

In [10]:

# Iteration G: Ensemble (GPU-optimized with minimal transfers)

print("Training Ensemble (Iter G)...")

horizons = sorted(train_df["horizon"].unique().to_list())
param_sets = [
    {"num_leaves": 31, "learning_rate": 0.05, "bagging_fraction": 0.8},
    {"num_leaves": 63, "learning_rate": 0.03, "bagging_fraction": 0.9},
]

preds_all_g = []

for h in horizons:
    t_h = train_df.filter(pl.col("horizon") == h)
    v_h = valid_df.filter(pl.col("horizon") == h)
    if t_h.height == 0: continue
    
    # Prepare features once on GPU
    X_valid_np = v_h.select(selected_features_f).fill_null(0).to_numpy()
    X_valid_gpu = cpu_to_gpu(X_valid_np)
    
    # Train models and collect predictions on GPU
    horizon_preds_gpu = []
    for p in param_sets:
        full_params = {"objective": "regression", "metric": "rmse", "verbose": -1, "n_jobs": -1, "device": "gpu", **p}
        model = train_lgb_model(t_h, v_h, selected_features_f, params=full_params)
        preds = model.predict(X_valid_np)
        horizon_preds_gpu.append(cpu_to_gpu(preds))
    
    # Average on GPU
    stacked_gpu = np.stack(horizon_preds_gpu)
    avg_preds_gpu = np.mean(stacked_gpu, axis=0)
    avg_preds = gpu_to_cpu(avg_preds_gpu)
    
    # Store with ID
    temp_df = v_h.select("id").with_columns(pl.Series("prediction", avg_preds))
    preds_all_g.append(temp_df)

if preds_all_g:
    submission = pl.concat(preds_all_g)
    submission.write_csv("submission_final_polars.csv")
    print("Saved submission_final_polars.csv")
    print(f"Submission shape: {submission.shape}")


Training Ensemble (Iter G)...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[103]	training's rmse: 0.212775	valid_1's rmse: 0.346027
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[181]	training's rmse: 0.16752	valid_1's rmse: 0.336707
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[91]	training's rmse: 0.217647	valid_1's rmse: 0.347597
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[162]	training's rmse: 0.171731	valid_1's rmse: 0.338075
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[95]	training's rmse: 0.218179	valid_1's rmse: 0.356681
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[147]	training's rmse: 0.178429	valid_1's rmse: 0.341759
Training until validation scores don't improve for 50 r